In [15]:
import pandas as pd

# Load the first sheet of the Excel file into a DataFrame
data_first_sheet = pd.read_excel("FantaSPL_V2.2.xlsx", sheet_name=0)

# Rename the columns
column_mapping = {
    "Data": "Date",
    "Tipo Partita": "Match Type",
    "Bianchi": "Team A Score",
    "Scuri": "Team B Score",
    "Squadra vincente": "Winning Team",
    "Numero Goal": "Number of Goals",
    "Goal Check": "Goal Check",
    "Numero Goal Punti": "Goal Points",
    "Autogoal": "Own Goals",
    "Costo per Partita": "Cost per Match",
    "Effettivo": "Actual Cost"
}
data_first_sheet.rename(columns=column_mapping, inplace=True)

# Extracting the relevant columns and create an explicit copy immediately
user_input_table = data_first_sheet[["Date", "Match Type", "Team A Score", "Team B Score"]].copy()

# Modify the 'Winning Team' column
user_input_table["Winning Team"] = user_input_table.apply(
    lambda row: "A" if row["Team A Score"] > row["Team B Score"] else ("B" if row["Team A Score"] < row["Team B Score"] else "X"), axis=1)

# Add the 'Total Score' column
user_input_table["Total Score"] = user_input_table["Team A Score"] + user_input_table["Team B Score"]

# Display the first few rows of the updated table
print(user_input_table.head())


        Date  Match Type  Team A Score  Team B Score Winning Team  Total Score
0 2023-01-12          14            18            13            A           31
1 2023-01-19          22             4             7            B           11
2 2023-01-26          16             5             8            B           13
3 2023-02-02          14             8            13            B           21
4 2023-02-09          14             6            11            B           17
